In [1]:
pip install -U keras-tuner

     |████████████████████████████████| 98 kB 3.9 MB/s 


In [16]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [17]:
mnist = tf.keras.datasets.mnist

(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train,x_test = x_train/255.0 , x_test/255.0

print("x_train.shape:",x_train.shape)
print("x_test.shape:",x_test.shape)

x_train.shape: (60000, 28, 28)
x_test.shape: (10000, 28, 28)


In [18]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

def build_model(hp):
  model = keras.Sequential()
  model.add(layers.Flatten(input_shape=(28,28)))

  model.add(layers.Dense(units=hp.Int('units',min_value=32,max_value=512,step=128),activation='relu'))
  model.add(layers.Dense(10,activation='softmax'))

  model.compile(
      optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-4])),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
  )
  return model

In [19]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld'
)

INFO:tensorflow:Reloading Oracle from existing project my_dir/helloworld/oracle.json


In [20]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 128, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0001], 'ordered': True}


In [21]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 6 Complete [00h 03m 33s]
val_accuracy: 0.9635333220163981

Best val_accuracy So Far: 0.9654333392779032
Total elapsed time: 00h 13m 17s
INFO:tensorflow:Oracle triggered exit


In [22]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 416
learning_rate: 0.0001
Score: 0.9654333392779032
Trial summary
Hyperparameters:
units: 288
learning_rate: 0.01
Score: 0.9635333220163981
Trial summary
Hyperparameters:
units: 160
learning_rate: 0.01
Score: 0.9617666602134705
Trial summary
Hyperparameters:
units: 160
learning_rate: 0.0001
Score: 0.9538000027338663
Trial summary
Hyperparameters:
units: 32
learning_rate: 0.0001
Score: 0.9290333390235901


# **Add for loops and vary activation functions**

In [23]:
def build_model2(hp):

  model = tf.keras.Sequential()
  model.add(layers.Flatten(input_shape=(28,28)))

  for i in range(hp.Int('layers',2,6)):
    model.add(
        tf.keras.layers.Dense(
            units=hp.Int('units_'+str(i),50,100,step=10),
            activation=hp.Choice('act_'+str(i),['relu','sigmoid'])
    )
        )
  model.add(tf.keras.layers.Dense(10,activation='softmax'))  
  model.compile('adam','sparse_categorical_crossentropy',metrics=['accuracy'])
  return model


In [24]:
tuner2 = RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=1,
    directory='my_dir2',
    project_name='helloworld'
)

In [25]:
tuner2.search_space_summary()

Search space summary
Default search space size: 5
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 10, 'sampling': None}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 10, 'sampling': None}
act_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}


In [26]:
tuner2.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 4 Complete [00h 00m 42s]
val_accuracy: 0.9656000137329102

Best val_accuracy So Far: 0.9731000065803528
Total elapsed time: 00h 02m 46s
INFO:tensorflow:Oracle triggered exit


In [27]:
tuner2.results_summary()

Results summary
Results in my_dir2/helloworld
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
layers: 3
units_0: 70
act_0: relu
units_1: 90
act_1: relu
units_2: 50
act_2: relu
units_3: 60
act_3: relu
Score: 0.9731000065803528
Trial summary
Hyperparameters:
layers: 3
units_0: 70
act_0: sigmoid
units_1: 50
act_1: relu
units_2: 50
act_2: relu
Score: 0.9693999886512756
Trial summary
Hyperparameters:
layers: 4
units_0: 90
act_0: sigmoid
units_1: 90
act_1: sigmoid
units_2: 90
act_2: relu
units_3: 50
act_3: relu
Score: 0.9690999984741211
Trial summary
Hyperparameters:
layers: 4
units_0: 60
act_0: sigmoid
units_1: 70
act_1: sigmoid
units_2: 50
act_2: relu
units_3: 100
act_3: sigmoid
Score: 0.9656000137329102


In [ ]:
# https://www.youtube.com/watch?v=Clo1HKB50Ug&t=617s